In [5]:
import pandas as pd
import numpy as np
import requests 
import os
import json
import tweepy
import matplotlib.pyplot as plt
import re
from timeit import default_timer as timer
from PIL import Image
from io import BytesIO
%matplotlib inline

# Gathering Data

- Importing the provided dataset, WeRateDogs Twitter archive 

In [6]:
df1 = pd.read_csv('twitter-archive-enhanced.csv')

- Downloading the image predictions file and saving it

In [7]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(os.getcwd(),url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [8]:
image_preds = pd.read_csv('image-predictions.tsv',sep = '\t')

- Quering Twitter API to gather more data using the tweets' IDs from the given dataset

Authentication 

In [9]:
api_key = 'bEvYcqpcl2ECziqw4U3W1N0jp'
api_secret = 'alDpEo1pBVIhzg3GiGywVtdeum92CvnEyJStAxSKv517JtKSpQ'
access_token = '1309464445102497792-DMKXVn3hTLJIibT82viDnkENIcRbHH'
access_token_secret = 'DtlGO4vdtF0nPSmsbo1hvbuGntMJisnMoxx6nyZyuw88G'

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True,parser=tweepy.parsers.JSONParser())

In [ ]:
tweets_ids = df1.tweet_id.values
count = 0
fail_dict = {}
start = timer()
with open('tweets.json','w') as output:
    for tweet_id in tweets_ids:
        count += 1
        print(str(count) + ':' + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode = 'extended')
            print('success')
            json.dump(tweet,output)
            output.write('\n')
        except tweepy.TweepError as e:
            print('failed')
            fail_dict[tweet_id] = e
            pass
end = timer()
print(start - end)
print(fail_dict)

1:892420643555336193
success
2:892177421306343426
success
3:891815181378084864
success
4:891689557279858688
success
5:891327558926688256
success
6:891087950875897856
success
7:890971913173991426
success
8:890729181411237888
success
9:890609185150312448
success
10:890240255349198849
success
11:890006608113172480
success
12:889880896479866881
success
13:889665388333682689
success
14:889638837579907072
success
15:889531135344209921
success
16:889278841981685760
success
17:888917238123831296
success
18:888804989199671297
success
19:888554962724278272
success
20:888202515573088257
failed
21:888078434458587136
success
22:887705289381826560
success
23:887517139158093824
success
24:887473957103951883
success
25:887343217045368832
success
26:887101392804085760
success
27:886983233522544640
success
28:886736880519319552
success
29:886680336477933568
success
30:886366144734445568
success
31:886267009285017600
success
32:886258384151887873
success
33:886054160059072513
success
34:88598480001994752

- I found out that it's more efficient to save the data I downloaded from Twitter archive in JSON format, as it'll take fewer codes to load it into a panda dataframe

In [ ]:
df2 = pd.read_json('tweets.json', lines=True)

# Assesing Data

In [ ]:
df1.info()

In [ ]:
df1.sample(25)

In [ ]:
df1['name'].value_counts()

In [ ]:
df1.duplicated().sum()

In [ ]:
df1['rating_denominator'].value_counts()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.rating_numerator.value_counts()

In [ ]:
df1[df1['rating_numerator'] == 0]

In [ ]:
df2.info()

In [ ]:
image_preds.info()

In [ ]:
image_preds.sample(25)

In [ ]:
image_preds.iloc[1640]

# Quality Issues for 
## DataFrame 1 (given dataset)
- Some of the tweets are retweets
- Some tweets are replies to another tweets
- Some tweets no longer exist
- Some values of numerators are way high like (420,666,1970,...)
- Some values of denominators are more than 10
- Some tweets' URLs are missing
- Some columns are not needed like ('retweeted_timestamp', 'source')
- Some dogs are missing their dog stages
- Some dogs names are None instead of NaN
- Some dog names are not added or aren't valid like (a, one,just,actually, etc..)

## DataFrame 2 (data downloaded using Twitter API)
- This dataframe almost has no quality issues
- Some columns aren't needed like (created at, lang, place, coordinates, source,..)

## Image Predictions 
- The guesses needs investigating to extract dog breeds and merge it into the master data frame
# Tidiness issues

- Column headers are values, not variable names (dog 'stage') in DataFrame1
- The image predictions file should be merged with DataFrame1
- Different names for columns for the same variables in the dataframes (tweet_id in df1, id in df2)
- Timestamp can be broken into 3 columns (hour ,day ,month) to analyze when did the account got the most interactions.

# Cleaning

In [ ]:
df1_c = df1.copy()
df2_c = df2.copy()
image_preds_c = image_preds.copy()

- Some of the tweets are retweets and some are replies
#### Define
- By using retweeted_status_id and in_reply_to_status_id columns in dataframe1, we can eliminate the retweets and eliminating quote tweets as well which are tweets retweeted with a quote in dataframe2

#### Code

In [ ]:
df1_c = df1_c[df1_c.retweeted_status_id.isnull()]
df1_c = df1_c[df1_c.in_reply_to_status_id.isnull()]
df2_c = df2_c[df2_c.retweeted_status.isnull()]
df2_c = df2_c[df2_c.in_reply_to_status_id.isnull()]
df2_c = df2_c[df2_c.quoted_status_id.isnull()]
df2_c = df2_c[~df2_c.is_quote_status]

#### Test

In [ ]:
df1_c.info()

In [ ]:
df2_c.info()

In [ ]:
df1_c.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp',
            'in_reply_to_status_id','in_reply_to_user_id'], axis= 1, inplace=True)
df2_c.drop(['in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id',
           'in_reply_to_user_id_str','in_reply_to_screen_name','retweeted_status',
            'quoted_status_id','quoted_status_id_str','quoted_status_permalink','quoted_status','is_quote_status'],axis=1,inplace=True)

In [ ]:
df1_c = df1_c.reset_index(drop=True)
df2_c = df2_c.reset_index(drop=True)

In [ ]:
df1_c.info()

In [ ]:
df2_c.info()

#### Define
- some rating numerator have way high numbers
- some denominators have numbers higher than 10
- treated the rating numerators by extracting the rating from the text using regular expression and correcting the whole column
- treated the rating denominator by using the assign function of pandas library

#### Code

In [ ]:
df1_c['rating_numerator'] = df1_c.text.str.extract(r"(\d{1,2}\/\d{2})")[0].str[:2].str.replace('/','')

In [ ]:
df1_c = df1_c.assign(rating_denominator = '10')

#### Testing

In [ ]:
df1_c.rating_denominator.value_counts()

In [ ]:
df1_c.rating_numerator.value_counts()

#### Define
- After checking tweet object on developer website, Some columns are not needed in dataframe2

#### Code

In [ ]:
df2_c.info()

In [ ]:
df2_c.drop(['id_str','truncated','place','lang','extended_entities','user','geo',
            'favorited','retweeted','possibly_sensitive','possibly_sensitive_appealable','coordinates',
            'source','contributors','entities'],axis=1,inplace=True )

#### Define
- Some dogs don't have dog stages

#### Code
- Gathering dog stages using regular expression from text

In [ ]:
df2_c['puppo'] = df2_c.full_text.str.extract(r"(puppo)")
df2_c['doggo'] = df2_c.full_text.str.extract(r"(doggo)")
df2_c['floofer'] = df2_c.full_text.str.extract(r"(floofer)")
df2_c['pupper'] = df2_c.full_text.str.extract(r"(pupper)")

In [ ]:
df1_c.drop(['puppo','doggo','floofer','pupper'],axis=1,inplace=True)

#### Testing

In [ ]:
df2_c[['pupper','doggo','puppo','floofer']].info()

#### Define
- Some dogs have no names or invalid names like (a,actually,..)

#### Code
- checking for the different patterns the name is usually written in tweets

In [ ]:
for text in df1_c[df1_c.name=='None'].text:
    print(text)

In [ ]:
for text in df1_c[df1_c.name == 'a'].text:
    print(text)

- Some tweets got names after "named", other got name after "His name is" and "Her name is" where the first letter is case sensitive
- Extracting the names from the tweets' text using regex

In [ ]:
df1_c['name_1'] = df1_c.text.str.extract(r"(named)+(.*?\.)")[1].str.replace('.','')

In [ ]:
df1_c['name_2'] = df1_c.text.str.extract(r"(his|His|her|Her)+( name is)+(.*?\.)")[2].str.replace('.','')

In [ ]:
df1_c.name_1.fillna(df1_c.name_2,inplace=True)

In [ ]:
df1_c.name_1.fillna(df1_c.name,inplace=True)

In [ ]:
df1_c.drop(['name','name_2'],axis=1,inplace=True)

In [ ]:
df1_c.rename(columns = {'name_1':'name'},inplace=True)

#### Testing

In [ ]:
df1_c.name.value_counts()

#### Define
- some invalid names still exits

#### Code

In [ ]:
df1_c.loc[df1_c.name == 'a','name'] = np.nan
df1_c.loc[df1_c.name == 'None','name'] = np.nan

#### Testing

In [ ]:
df1_c.name.value_counts()

In [ ]:
df2_c.info()

#### Define
- more than one variable in created_at column and changing the column to timestamp for better and familiar description

#### Code

In [ ]:
df2_c['day'] = df2_c.created_at.dt.day_name()

In [ ]:
df2_c['hour'] = df2_c.created_at.dt.hour

In [ ]:
df2_c['month'] = df2_c.created_at.dt.month

In [ ]:
df2_c['year'] = df2_c.created_at.dt.year

#### Testing

In [ ]:
df2_c.sample(20)

- Dropping some unnecessary columns

In [ ]:
df2_c.drop(['display_text_range','full_text'],inplace=True,axis=1)

In [ ]:
df1_c.drop(['source','timestamp'],inplace=True,axis=1)

- renaming some columns for merging

In [ ]:
df2_c.rename(columns = {'created_at':'timestamp','id':'tweet_id'},inplace=True)

In [ ]:
df1_c.info()

In [ ]:
df2_c.info()

#### Define
- The two dataframes (given, gathered) should be merged together

#### Code

In [ ]:
master_df = pd.merge(df1_c,df2_c,how='inner',on='tweet_id')

#### Testing

In [ ]:
master_df.sample(10)

#### Define 
- Dog stages are columns' headers instead of values

#### Code
- replacing NaN values with empty strings to facilitate merging columns

In [ ]:
master_df.doggo.fillna(value='',inplace=True)
master_df.puppo.fillna(value='',inplace=True)
master_df.floofer.fillna(value='',inplace=True)
master_df.pupper.fillna(value='',inplace=True)

In [ ]:
master_df['dog_stage'] = master_df['puppo'] + master_df['doggo'] + master_df['floofer'] + master_df['pupper']

In [ ]:
master_df.drop(['puppo','doggo','floofer','pupper'], inplace=True,axis=1)

In [ ]:
master_df.dog_stage.value_counts()

In [ ]:
master_df.loc[master_df.dog_stage == 'doggopupper','dog_stage'] = 'doggo-pupper'

In [ ]:
master_df.loc[master_df.dog_stage == 'puppodoggo','dog_stage'] = 'puppo-doggo'

In [ ]:
master_df.dog_stage.value_counts()

In [ ]:
master_df.dog_stage.replace({'':np.nan},inplace=True)

#### Define
- Upon investigating some images in the image predictions file, some images are more suitable to the 2nd guess as the dog wasn't as clear or a bit far in the image so the 1st guess guessing whether it's a dog or not is false.

In [ ]:
image_preds_c['breed'] = image_preds_c[image_preds_c.p1_dog].p1
image_preds_c.breed.value_counts()

In [ ]:
image2 = image_preds_c[~image_preds_c.p1_dog]
breed2 = image2[image2.p2_dog].p2
image_preds_c.breed.fillna(breed2,inplace=True)
image_preds_c.breed.value_counts()

In [ ]:
image3 = image2[~image2.p2_dog]
breed3 = image3[image3.p3_dog].p3
image_preds_c.breed.fillna(breed3,inplace=True)
image_preds_c.breed.value_counts()

In [ ]:
impr = image_preds_c[['tweet_id','jpg_url','img_num','breed']]

In [ ]:
total = pd.merge(master_df,impr,on='tweet_id',how='inner')

#### Testing
- checking the tweet url and the image url to see if they're the same image and that merging was successful

In [ ]:
total.iloc[58].jpg_url

In [ ]:
total.iloc[58].expanded_urls

# Storing Cleaned Dataset

In [ ]:
total.to_csv('twitter_archive_master.csv')

# Analysis and Visualization

In [ ]:
df = pd.read_csv('twitter_archive_master.csv')

- Analyzing data of dog breed and favorite and retweet counts to get informed on which dog breed is more likable
- It appears that Bedlington is the most likable

In [ ]:
df.groupby('breed')[['favorite_count','retweet_count']].mean().sort_values(by = ['favorite_count','retweet_count'])

In [ ]:
df.groupby('breed')[['favorite_count','retweet_count']].mean().sort_values(by = ['favorite_count','retweet_count']).plot(figsize=(25,10),kind='bar')

- dog stage data and retweet and favorite counts
- puppo-doggo is the most likable

In [ ]:
df.groupby('dog_stage')[['favorite_count','retweet_count']].mean().sort_values(by=['favorite_count','retweet_count'])

In [ ]:
df.groupby('dog_stage')[['favorite_count','retweet_count']].mean().sort_values(by=['favorite_count','retweet_count']).plot(kind='bar',figsize=(8,6))

- Analyzing interaction based on day
- Wednesday is the day with the most interaction

In [ ]:
df.groupby('day')[['favorite_count','retweet_count']].mean()

In [ ]:
df.groupby('day')[['favorite_count','retweet_count']].mean().plot(kind='bar',figsize=(8,6))

- Analyzing interaction based on hour
- 6 am is the most hour with interaction

In [ ]:
df.groupby('hour')[['favorite_count','retweet_count']].mean()

In [ ]:
df.groupby('hour')[['favorite_count','retweet_count']].mean().plot(kind='bar',figsize=(8,6))

- Analyzing interaction based on month
- June is the month with most interaction

In [ ]:
df.groupby('month')[['favorite_count','retweet_count']].mean()

In [ ]:
df.groupby('month')[['favorite_count','retweet_count']].mean().plot(kind='bar',figsize=(8,6))

- Visualizing interaction of the given period based on the number of likes and retweets

In [ ]:
df.groupby(['year','month'])[['retweet_count','favorite_count']].mean().plot(figsize=(8,6))

- Analyzing the number of tweets in the given period showed us that the account was focused on tweeting attractive tweets at first to gain followers then lowered its tweets 

In [ ]:
total.groupby(['year','month'])[['tweet_id']].count().plot(figsize=(8,6))